In [1]:
# Load the dependencies
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

#visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
#ignore warnings
import warnings
warnings.filterwarnings('ignore')

#os

import os


## The files are a snapshot of street-level crime and stop and search information across the years from 2022 till 2024 for different Police Forces in England. It is for 43 geographic police forces in  England and Wales, the British Transport Police, the Police Service of Northern Ireland and the Ministry of Justice. All data is anonymised.##

reference: https://data.police.uk/about/#columns

Another file called force boundaries shows the geographic data using KML format(used to display geographic data in an earth browser like Google earth)

**Columns are as follows:**

Field	Meaning
- Reported by:      The force that provided the data about the crime.
- Falls within:     At present, also the force that provided the data about the crime. This is currently being looked into and is likely to change in the near future.
- Longitude and Latitude:	    The anonymised coordinates of the crime. See Location Anonymisation for more information.
- LSOA code and LSOA name:  	References to the Lower Layer Super Output Area that the anonymised point falls into, according to the LSOA boundaries provided by the Office for National Statistics.
- Crime type:   	One of the crime types listed in the Police.UK FAQ.
- Last outcome category:    	A reference to whichever of the outcomes associated with the crime occurred most recently. For example, this crime's 'Last outcome category' would be 'Formal action is not in the public interest'.
Context	A field provided for forces to provide additional human-readable data about individual crimes. Currently, for newly added CSVs, this is always empty.


Individual crime and anti-social behaviour (ASB) incidents, including street-level location information and subsequent police and court outcomes associated with the crime are of interest.

**data issues:**

- lattitue and longitude locations represent the approximate location not exact.
- Nearly impossible to track the crime through the entire criminal justice process due to lack of unique identifiers.
- Possible duplicate data for ASB and crime in 6 police forces.

**NB:  A "Lower Layer Super Output Area (LSOA) boundary" refers to the geographical perimeter of a small, defined area used in the UK census, typically containing between 1,000 and 3,000 people, made up of a group of smaller "Output Areas" (OAs), and considered the lowest level of geographical detail for census statistics within a region; essentially, it's a boundary that delineates a neighborhood-like area with a relatively consistent population size for data analysis**


In [2]:
import os
import duckdb

BASE_DIR = "./data"
if not os.path.isdir(BASE_DIR):
    raise Exception("The directory does not exist. Please check your path.")

file_paths = {}
for ym in sorted(os.listdir(BASE_DIR)):
    ym_path = os.path.join(BASE_DIR, ym)
    if os.path.isdir(ym_path):
        csvs = [
            os.path.join(ym_path, fn)
            for fn in os.listdir(ym_path)
            if fn.lower().endswith(".csv")
        ]
        if csvs:
            file_paths[ym] = csvs

con = duckdb.connect()

In [3]:
def classify_from_duckdb(col_name, duck_type, summary):
    data_type = duck_type.upper()
    numeric_indicators = ("INT", "DOUBLE", "DECIMAL", "FLOAT")
    is_numeric = any(keyword in data_type for keyword in numeric_indicators)

    if is_numeric:
 
        min_value = summary.get("min")


        if min_value is None:
            return "Interval"

    
        lower_name = col_name.lower()
        if "lat" in lower_name or "lon" in lower_name:
            return "Ratio"

        if min_value >= 0:
            return "Ratio"
        else:
            return "Interval"


    unique_values = summary.get("unique", 0)
    if unique_values <= 2:
        return "Nominal (Binary)"


    ordinal_keywords = ("age", "range", "rank", "level", "grade")
    if any(keyword in col_name.lower() for keyword in ordinal_keywords):
        return "Ordinal"

    datetime_keywords = ("date", "time", "year", "month")
    if any(keyword in col_name.lower() for keyword in datetime_keywords):
        if "TIMESTAMP" in data_type or "DATE" in data_type:
            return "Interval"
        else:
            return "Nominal"
    return "Nominal"


In [4]:
schema_groups = {}
for month, paths in file_paths.items():
    for path in paths:
        # DESCRIBE returns a small table of column_name & column_type
        desc = con.execute(f"DESCRIBE SELECT * FROM read_csv_auto('{path}')").fetchdf()
        schema_key = tuple(desc["column_name"])
        schema_groups.setdefault(schema_key, []).append((month, path))

In [7]:
print(cols)


('Crime ID', 'Month', 'Reported by', 'Falls within', 'Longitude', 'Latitude', 'Location', 'LSOA code', 'LSOA name', 'Crime type', 'Last outcome category', 'Context')


In [8]:
print(recs)

[('2022-01', './data\\2022-01\\2022-01-avon-and-somerset-street.csv'), ('2022-01', './data\\2022-01\\2022-01-bedfordshire-street.csv'), ('2022-01', './data\\2022-01\\2022-01-btp-street.csv'), ('2022-01', './data\\2022-01\\2022-01-cambridgeshire-street.csv'), ('2022-01', './data\\2022-01\\2022-01-cheshire-street.csv'), ('2022-01', './data\\2022-01\\2022-01-city-of-london-street.csv'), ('2022-01', './data\\2022-01\\2022-01-cleveland-street.csv'), ('2022-01', './data\\2022-01\\2022-01-cumbria-street.csv'), ('2022-01', './data\\2022-01\\2022-01-derbyshire-street.csv'), ('2022-01', './data\\2022-01\\2022-01-devon-and-cornwall-street.csv'), ('2022-01', './data\\2022-01\\2022-01-dorset-street.csv'), ('2022-01', './data\\2022-01\\2022-01-durham-street.csv'), ('2022-01', './data\\2022-01\\2022-01-dyfed-powys-street.csv'), ('2022-01', './data\\2022-01\\2022-01-essex-street.csv'), ('2022-01', './data\\2022-01\\2022-01-gloucestershire-street.csv'), ('2022-01', './data\\2022-01\\2022-01-gwent-stree

In [11]:
print(schema_groups.items())

dict_items([(('Type', 'Date', 'Part of a policing operation', 'Policing operation', 'Latitude', 'Longitude', 'Gender', 'Age range', 'Self-defined ethnicity', 'Officer-defined ethnicity', 'Legislation', 'Object of search', 'Outcome', 'Outcome linked to object of search', 'Removal of more than just outer clothing'), [('2022-01', './data\\2022-01\\2022-01-avon-and-somerset-stop-and-search.csv'), ('2022-01', './data\\2022-01\\2022-01-bedfordshire-stop-and-search.csv'), ('2022-01', './data\\2022-01\\2022-01-btp-stop-and-search.csv'), ('2022-01', './data\\2022-01\\2022-01-cambridgeshire-stop-and-search.csv'), ('2022-01', './data\\2022-01\\2022-01-cheshire-stop-and-search.csv'), ('2022-01', './data\\2022-01\\2022-01-city-of-london-stop-and-search.csv'), ('2022-01', './data\\2022-01\\2022-01-cleveland-stop-and-search.csv'), ('2022-01', './data\\2022-01\\2022-01-cumbria-stop-and-search.csv'), ('2022-01', './data\\2022-01\\2022-01-derbyshire-stop-and-search.csv'), ('2022-01', './data\\2022-01\\2

In [12]:
print(grp_id)

2


In [ ]:
import duckdb

con = duckdb.connect()
con.execute("SET max_expression_depth TO 5000;")

def quote_identifier(col: str) -> str:
    """Wrap a column name in double quotes escaping any internal quotes."""
    escaped = col.replace('"', '""')
    return f'"{escaped}"'

for grp_id, (cols, recs) in enumerate(schema_groups.items(), start=1):
    print(f"Group {grp_id}: {len(recs)} files share these columns:")
    print(cols)

    first_path = recs[0][1]
    desc = con.execute(
        f"DESCRIBE SELECT * FROM read_csv_auto('{first_path}')"
    ).fetchdf()
    column_types = dict(zip(desc.column_name, desc.column_type))

   
    total_rows = 0
    stats = {col: {"nulls": 0, "min": None, "max": None} for col in cols}

    for month, path in recs:
        exprs = ["COUNT(*) AS cnt"]
        for i, col in enumerate(cols):
            q = quote_identifier(col)
            exprs += [
                f"COUNT_IF({q} IS NULL)    AS nulls_{i}",
                f"MIN({q})                 AS min_{i}",
                f"MAX({q})                 AS max_{i}",
            ]

        sql = f"SELECT {', '.join(exprs)} FROM read_csv_auto('{path}')"
        row = con.execute(sql).fetchone()
        total_rows += row[0]

        # unpack and accumulate
        idx = 1
        for i, col in enumerate(cols):
            nulls, mn, mx = row[idx], row[idx+1], row[idx+2]
            idx += 3
            stats[col]["nulls"] += nulls
            if mn is not None and (stats[col]["min"] is None or mn < stats[col]["min"]):
                stats[col]["min"] = mn
            if mx is not None and (stats[col]["max"] is None or mx > stats[col]["max"]):
                stats[col]["max"] = mx

    # 4) print out
    print(f"\nTotal rows across group: {total_rows}\n")
    header = (
        f"{'Column':<30}{'Type':<15}"
        f"{'Count':>8}{'Nulls':>8}"
        f"{'Min':>12}{'Max':>12}{'Scale':>10}"
    )
    print(header)
    print("-" * len(header))

    for col in cols:
        ct = column_types.get(col, "UNKNOWN")
        scale = classify_from_duckdb(col, ct, {"min": stats[col]["min"]})
        print(
            f"{col:<30}{ct:<15}"
            f"{total_rows:>8}{stats[col]['nulls']:>8}"
            f"{str(stats[col]['min']):>12}{str(stats[col]['max']):>12}"
            f"{scale:>10}"
        )

print("\nAll groups analyzed—each file streamed one at a time, so you stay under your memory cap!")


Group 1: 1424 files share these columns:
('Type', 'Date', 'Part of a policing operation', 'Policing operation', 'Latitude', 'Longitude', 'Gender', 'Age range', 'Self-defined ethnicity', 'Officer-defined ethnicity', 'Legislation', 'Object of search', 'Outcome', 'Outcome linked to object of search', 'Removal of more than just outer clothing')

Total rows across group: 1408135

Column                        Type              Count   Nulls         Min         Max     Scale
-----------------------------------------------------------------------------------------------
Type                          VARCHAR         1408135       0Person and Vehicle searchVehicle searchNominal (Binary)
Date                          TIMESTAMP       1408135       02022-01-01 00:00:002024-12-31 23:54:48Nominal (Binary)
Part of a policing operation  VARCHAR         1408135  587533       False        TrueNominal (Binary)
Policing operation            VARCHAR         1408135 1408135        None        NoneNominal (B

In [16]:
# drop “Policing operation” from schema_groups ──
new_schema_groups = {}
for cols, recs in schema_groups.items():
    filtered_cols = tuple(c for c in cols if c != "Policing operation")
    new_schema_groups.setdefault(filtered_cols, []).extend(recs)


schema_groups = new_schema_groups
for grp_id, (cols, recs) in enumerate(schema_groups.items(), start=1):
    print(f"Group {grp_id}: {len(recs)} files share these columns:")
    print(cols)


Group 1: 1424 files share these columns:
('Type', 'Date', 'Part of a policing operation', 'Latitude', 'Longitude', 'Gender', 'Age range', 'Self-defined ethnicity', 'Officer-defined ethnicity', 'Legislation', 'Object of search', 'Outcome', 'Outcome linked to object of search', 'Removal of more than just outer clothing')
Group 2: 1565 files share these columns:
('Crime ID', 'Month', 'Reported by', 'Falls within', 'Longitude', 'Latitude', 'Location', 'LSOA code', 'LSOA name', 'Crime type', 'Last outcome category', 'Context')


In [18]:
con = duckdb.connect()
con.execute("SET max_expression_depth TO 5000;")

# expected columns
STOP_SEARCH_COLUMNS = {
    'Type','Date','Part of a policing operation',
    'Latitude','Longitude','Gender','Age range',
    'Self-defined ethnicity','Officer-defined ethnicity',
    'Legislation','Object of search','Outcome',
    'Outcome linked to object of search',
    'Removal of more than just outer clothing'
}
CRIME_COLUMNS = {
    'Crime ID','Month','Reported by','Falls within',
    'Longitude','Latitude','Location',
    'LSOA code','LSOA name','Crime type',
    'Last outcome category','Context'
}


def quote(col):
    return '"' + col.replace('"','""') + '"'

stop_paths = []
crime_paths = []
STOP_SEARCH_MISMATCH = {}
CRIME_MISMATCH = {}


In [19]:
# Split schema_groups by stop_search and crime_columns
for grp_id, (cols, recs) in enumerate(schema_groups.items(), start=1):
    colset = set(cols)
    if 'Crime ID' in colset:
        for month, path in recs:
            key = f"{month}_grp{grp_id}"
            missing = CRIME_COLUMNS - colset
            extra   = colset - CRIME_COLUMNS
            if missing or extra:
                CRIME_MISMATCH[key] = {'missing': missing, 'extra': extra}
            crime_paths.append((month, path))
    elif 'Type' in colset:
        for month, path in recs:
            key = f"{month}_grp{grp_id}"
            missing = STOP_SEARCH_COLUMNS - colset
            extra   = colset - STOP_SEARCH_COLUMNS
            if missing or extra:
                STOP_SEARCH_MISMATCH[key] = {'missing': missing, 'extra': extra}
            stop_paths.append((month, path))
    else:
        print(f"Unclassified grp{grp_id}: columns {cols}")


In [ ]:


import os

BASE_DIR = "./data"
stop_pattern  = os.path.join(BASE_DIR, "*", "*-stop-and-search.csv")
crime_pattern = os.path.join(BASE_DIR, "*", "*-street.csv")


stop_cols   = sorted(STOP_SEARCH_COLUMNS)
stop_select = ", ".join(
    f"COALESCE({quote_identifier(c)}, 'unknown') AS {quote_identifier(c)}"
    for c in stop_cols
)
crime_cols   = sorted(CRIME_COLUMNS)
crime_select = ", ".join(
    f"COALESCE({quote_identifier(c)}, 'unknown') AS {quote_identifier(c)}"
    for c in crime_cols
)


con.execute(f"""
CREATE OR REPLACE VIEW stop_and_search AS
SELECT {stop_select}
FROM read_csv_auto('{stop_pattern}', union_by_name=TRUE);
""")

con.execute(f"""
CREATE OR REPLACE VIEW crime AS
SELECT {crime_select}
FROM read_csv_auto('{crime_pattern}', union_by_name=TRUE);
""")


ss_count = con.execute("SELECT COUNT(*) FROM stop_and_search").fetchone()[0]
c_count  = con.execute("SELECT COUNT(*) FROM crime").fetchone()[0]

print("stop_and_search rows:", ss_count)
print("crime rows:         ", c_count)
print("\nStop-and-search mismatches:", STOP_SEARCH_MISMATCH)
print("Crime mismatches:        ", CRIME_MISMATCH)


stop_and_search rows: 1408135
crime rows:          18141017

Stop-and-search mismatches: {}
Crime mismatches:         {}


In [26]:
NUMERIC_COLS = {'Latitude', 'Longitude'}


def build_select(columns):
    parts = []
    for c in sorted(columns):
        qc = quote_identifier(c)
        if c in NUMERIC_COLS:
            # leave numeric columns as is
            parts.append(f"{qc}")
        else:
            parts.append(f"COALESCE({qc}, 'unknown') AS {qc}")
    return ", ".join(parts)

stop_select = build_select(STOP_SEARCH_COLUMNS)
crime_select = build_select(CRIME_COLUMNS)


con.execute(f"""
CREATE OR REPLACE VIEW stop_and_search AS
SELECT {stop_select}
FROM read_csv_auto('{stop_pattern}', union_by_name=TRUE);
""")

con.execute(f"""
CREATE OR REPLACE VIEW crime AS
SELECT {crime_select}
FROM read_csv_auto('{crime_pattern}', union_by_name=TRUE);
""")


con.execute("""
COPY stop_and_search
TO 'stop_and_search_data.csv'
(HEADER, DELIMITER ',');
""")
con.execute("""
COPY crime
TO 'crime_data.csv'
(HEADER, DELIMITER ',');
""")
print(" CSVs written: stop_and_search_data.csv, crime_data.csv")


print("\nRemaining NULL counts for numeric columns:")
for view in ("stop_and_search", "crime"):
    print(f"\n{view}:")
    for numcol in NUMERIC_COLS:
        cnt = con.execute(f"""
            SELECT COUNT_IF({quote_identifier(numcol)} IS NULL)
            FROM {view};
        """).fetchone()[0]
        print(f"  {numcol}: {cnt}")


✔️ CSVs written: stop_and_search_data.csv, crime_data.csv

Remaining NULL counts for numeric columns:

stop_and_search:
  Latitude: 179404
  Longitude: 179404

crime:
  Latitude: 297009
  Longitude: 297009
